Importing Pandas for using dataframes to extract data.
Importing bs4 for text parsing

In [19]:
import pandas as pd
from bs4 import BeautifulSoup, Comment

Reading in rawdata of boxscores

In [20]:
numbers=[16,17,18,19,20,21,22,23]
All_dfs = []
for i in numbers:
    All_dfs.append(pd.read_csv(f'/Users/migcord/Springboard/Capstone/Spiders/NBA/data/20{i}_data.csv'))
s16,s17,s18,s19,s20,s21,s22,s23 = All_dfs[0],All_dfs[1],All_dfs[2],All_dfs[3],All_dfs[4],All_dfs[5],All_dfs[6],All_dfs[7]


Defining  functions for extracting data

Parsing function removes unnessecary data takes in raw tect and returns beautful soup data type

In [ ]:

def parsing(unfiltered):
    soup = BeautifulSoup(unfiltered,'lxml')
    [soups.decompose() for soups in soup.select('tr.over_header')]
    [soups.decompose() for soups in soup.select('tr.thead')]
    return soup

Linescore function 
takes in soup object
Finds all comments from soup then extracts table from the correct comment
Returns df filled with linescore

In [ ]:

def line_score(soup):
    comments = soup.find_all(string=lambda text: isinstance(text, Comment))
    linescore= None
    for comment in comments:
        if 'line_score' in comment:
            linescore = pd.read_html(str(comment),attrs={'id': 'line_score'})[0]
            break
    cols =list(linescore.columns)
    cols[0],cols[-1] = 'team','total'
    linescore.columns = cols
    linescore = linescore[['team','total']]
    return linescore

ReadStats function
finds box table and returns it.

In [ ]:

def readstats(soup,team,stat):
    box = pd.read_html(str(soup),attrs={'id':f'box-{team}-game-{stat}'},index_col=0)[0]
    box= box.apply(pd.to_numeric, errors='coerce')
    return box

Grabs date

In [21]:
def get_date(soup):
    try:
        date = str(soup.select('div.scorebox_meta > div:first-child')[0]).removeprefix('<div>')
        date = date.removesuffix('</div>')
        date = pd.to_datetime(date)
    except:
        return None
    return date

CreatingSeasonStats
takes in pd series 
parses it and extracts linescore/boxscores(advanced and basic)
it then combines both teams totals and creates a game df combining both team stats. 
returns df of the full game

In [22]:
def CreatingSeasonStats(boxscores):
    base_cols =None
    games = []
    for boxscore in boxscores:
        soup = parsing(boxscore)
        date =get_date(soup)
        linescore= line_score(soup)
        summaries = []
        for team in linescore['team']:
    
            basic = readstats(soup,team,'basic')
            advanced = readstats(soup,team,'advanced')
    
            total = pd.concat([basic.iloc[-1,:],advanced.iloc[-1,:]])
            total.index = total.index.str.lower()+'_total'
    
            maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1,:].max()])
            maxes.index = maxes.index.str.lower()+'_max'
    
            game_summary = pd.concat([total,maxes])
    
            if base_cols is None:
                base_cols = list(game_summary.index.drop_duplicates(keep='first'))
                base_cols = [consistent for consistent in base_cols if 'bpm' not in consistent]
        
            game_summary = game_summary[base_cols]
            summaries.append(game_summary)
        summary = pd.concat(summaries,axis=1).T

        game =pd.concat([summary,linescore],axis=1)
        game['date'] = date
        game['home'] = [0,1]
        game_opp = game.iloc[::-1].reset_index()
        game_opp.columns += "_opp"
        full_game = pd.concat([game,game_opp], axis=1)
        full_game['won'] = full_game['total'] > full_game['total_opp']
        games.append(full_game)
        if len(games) % 100 == 0:
            print(len(games))
    print("finished!!")
    return pd.concat(games,ignore_index=True)

In [24]:
df_stats16 = CreatingSeasonStats(s16['Box_Score'])
df_stats17 = CreatingSeasonStats(s17['Box_Score'])
df_stats18 = CreatingSeasonStats(s18['Box_Score'])
df_stats19 = CreatingSeasonStats(s19['Box_Score'])
df_stats20 = CreatingSeasonStats(s20['Box_Score'])
df_stats21 = CreatingSeasonStats(s21['Box_Score'])
df_stats22 = CreatingSeasonStats(s22['Box_Score'])
df_stats23 = CreatingSeasonStats(s23['Box_Score'])


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
finished!!
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
finished!!
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
finished!!
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
finished!!
100
200
300
400
500
600
700
800
900
1000
1100
finished!!
100
200
300
400
500
600
700
800
900
1000
1100
1200
finished!!
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
finished!!
100
200
300
400
500
600
700
finished!!


In [25]:
df_stats16.to_csv('2016_Boxscores.csv')
df_stats17.to_csv('2017_Boxscores.csv')
df_stats18.to_csv('2018_Boxscores.csv')
df_stats19.to_csv('2019_Boxscores.csv')
df_stats20.to_csv('2020_Boxscores.csv')
df_stats21.to_csv('2021_Boxscores.csv')
df_stats22.to_csv('2022_Boxscores.csv')
df_stats23.to_csv('2023_Boxscores.csv')

In [29]:
AllBoxScores = pd.concat([df_stats16,df_stats17,df_stats18,df_stats19,df_stats20,df_stats21,df_stats22,df_stats23])

In [31]:
AllBoxScores.to_csv('AllBoxScores.csv')